In [ ]:
from src import loaders as loadfile
from src import chunking as ch
from src import embedding as embed
from src import vector_store_client as vstore
from qdrant_client import QdrantClient
from dotenv import load_dotenv
load_dotenv()

#Parametros:
chunk_size = 500
chunk_overlap = 100
folder_path = "../practicos-rag/data"


#Carga de documentos:
docs_from_folder = loadfile.load_documents_from_folder(folder_path)

#Chunking
splits = ch.chunk_text(docs_from_folder, chunk_size, chunk_overlap)
splits[:2]

ModuleNotFoundError: No module named 'chunking.document_splitter'

In [ ]:
# Crear una instancia de la clase personalizada
embedding_model = embed.CustomHuggingFaceEmbeddings(mode="sentence")

# Obtener la dimensión del vector
dimension = embedding_model.get_dimension()
print(f"La dimensión del vector es: {dimension}")

# Nombre de la colección
collection_name = "demo_collection2"
update = True

# Conectar al cliente Qdrant
client = QdrantClient(host="localhost", port=6333)
print('Cliente conectado')

vector_store = vstore.create_vector_store(client, collection_name, embedding_model, splits, dimension, update = True)

In [ ]:
from langchain_ollama import OllamaLLM
#!ollama pull llama3.2
llm = OllamaLLM(model="llama3.2")

In [ ]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
prompt.messages[0].prompt.template

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

retriever = vector_store.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# RAG chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
!ollama pull llama3.2

In [ ]:
rag_chain.invoke("Can you tell me what are the regulations for labeling of cacao and chocolate?")

In [ ]:
#!pip freeze > requirements.txt